In [5]:
import json
import os
import csv
import pandas as pd
import numpy as np

In [2]:
path = '../raw_data/'
hist = {}
second_hist = {}
fnames = []
for file in os.listdir(path):
    if file[-9:] != '_2048.out':
        continue
    id_ctr = 0
    hist_list = []
    with open(path+file) as f:
        lines = f.readlines()
        ctr = 0
        conf_values = ['id']
        while lines[ctr] != "~~~~~~~~~~Reuse Distance Profiler output~~~~~~~~~~\n":
            ctr += 1
        ctr += 1
        while lines[ctr] != "~~~~~~~~~~Reuse Distance Profiler output end~~~~~~~~~~\n":
            x, y, z = lines[ctr].split()
            conf_values.append(x)
            second_hist[x] = int(y)
            ctr += 1
        while len(lines) - 20 > ctr:
            while lines[ctr] != "~~~~~~~~~~Reuse Distance Profiler output~~~~~~~~~~\n":
                ctr += 1
            hist['id'] = file[:-8] + str(id_ctr)
            id_ctr += 1
            ctr += 1
            while lines[ctr] != "~~~~~~~~~~Reuse Distance Profiler output end~~~~~~~~~~\n":
                x, y, z = lines[ctr].split()
                hist[x] = int(y) - second_hist[x]
                second_hist[x] = int(y)
                ctr += 1
            hist_list.append(hist.copy())
    id_ctr -= 1       # Since we don't keep the last one
    fnames.append(file[:-9])
    print(f'Printing: '+'reuse_distances/'+file[:-9]+'.csv'+f', {len(hist_list)} rds')
    with open('reuse_distances/'+file[:-9]+'.csv', 'w', newline="") as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=conf_values)
        writer.writeheader()
        # writer.writerow(second_hist)    # Final/full histogram
        for hst in hist_list[:-1]:     # (last one's all 0 and first one has warmup instructions)
            writer.writerow(hst)        # In between histograms


Printing: reuse_distances/400.perlbench-41B.csv, 750 rds
Printing: reuse_distances/400.perlbench-50B.csv, 750 rds
Printing: reuse_distances/401.bzip2-226B.csv, 750 rds
Printing: reuse_distances/401.bzip2-277B.csv, 750 rds
Printing: reuse_distances/401.bzip2-38B.csv, 750 rds
Printing: reuse_distances/401.bzip2-7B.csv, 750 rds
Printing: reuse_distances/403.gcc-16B.csv, 750 rds
Printing: reuse_distances/403.gcc-17B.csv, 750 rds
Printing: reuse_distances/403.gcc-48B.csv, 750 rds
Printing: reuse_distances/410.bwaves-1963B.csv, 750 rds
Printing: reuse_distances/410.bwaves-2097B.csv, 750 rds
Printing: reuse_distances/410.bwaves-945B.csv, 750 rds
Printing: reuse_distances/416.gamess-875B.csv, 750 rds
Printing: reuse_distances/429.mcf-184B.csv, 750 rds
Printing: reuse_distances/429.mcf-192B.csv, 750 rds
Printing: reuse_distances/429.mcf-217B.csv, 750 rds
Printing: reuse_distances/429.mcf-22B.csv, 750 rds
Printing: reuse_distances/429.mcf-51B.csv, 750 rds
Printing: reuse_distances/433.milc-127B.

Printing: reuse_distances/638.imagick_s-824B.csv, 750 rds
Printing: reuse_distances/641.leela_s-1052B.csv, 750 rds
Printing: reuse_distances/641.leela_s-1083B.csv, 750 rds
Printing: reuse_distances/641.leela_s-149B.csv, 750 rds
Printing: reuse_distances/641.leela_s-334B.csv, 750 rds
Printing: reuse_distances/641.leela_s-602B.csv, 750 rds
Printing: reuse_distances/641.leela_s-800B.csv, 750 rds
Printing: reuse_distances/641.leela_s-862B.csv, 750 rds
Printing: reuse_distances/644.nab_s-12459B.csv, 750 rds
Printing: reuse_distances/644.nab_s-12521B.csv, 750 rds
Printing: reuse_distances/644.nab_s-5853B.csv, 750 rds
Printing: reuse_distances/644.nab_s-7928B.csv, 750 rds
Printing: reuse_distances/644.nab_s-9322B.csv, 750 rds
Printing: reuse_distances/648.exchange2_s-1227B.csv, 750 rds
Printing: reuse_distances/648.exchange2_s-1247B.csv, 750 rds
Printing: reuse_distances/648.exchange2_s-1511B.csv, 750 rds
Printing: reuse_distances/648.exchange2_s-1699B.csv, 750 rds
Printing: reuse_distances/6

In [3]:
import pandas as pd
import numpy as np

In [4]:
path = '../ce_raw_data/raw_data/'
id_ctr = 0
columns = ['id','32','64','128','256','512','1024','2048','4096','8192']
l1d_full = pd.DataFrame()
l2c_full = pd.DataFrame()
llc_full = pd.DataFrame()
l1d_accesses_full = pd.DataFrame()
l2c_accesses_full = pd.DataFrame()
llc_accesses_full = pd.DataFrame()
l1d_misses_full = pd.DataFrame()
l2c_misses_full = pd.DataFrame()
llc_misses_full = pd.DataFrame()
for file in fnames:
    l1d_small = pd.DataFrame(0., index=np.arange(749), columns=columns)
    l2c_small = pd.DataFrame(0., index=np.arange(749), columns=columns)
    llc_small = pd.DataFrame(0., index=np.arange(749), columns=columns)
    l1d_accesses = pd.DataFrame(0, index=np.arange(749), columns=columns)
    l2c_accesses = pd.DataFrame(0, index=np.arange(749), columns=columns)
    llc_accesses = pd.DataFrame(0, index=np.arange(749), columns=columns)
    l1d_misses = pd.DataFrame(0, index=np.arange(749), columns=columns)
    l2c_misses = pd.DataFrame(0, index=np.arange(749), columns=columns)
    llc_misses = pd.DataFrame(0, index=np.arange(749), columns=columns)
    # print(file)
    for c in columns[1:]:
        fname = file + '_' + c + '.out'
        checker = False
        with open(path+fname) as f:
            lines = f.readlines()
            ctr = 0
            prev_misses_l1d = 0.
            prev_accesses_l1d = 0.
            prev_misses_l2c = 0.
            prev_accesses_l2c = 0.
            prev_misses_llc = 0.
            prev_accesses_llc = 0.
            for line in lines:
                if line != "~~~~~~~~~~Reuse Distance Profiler output~~~~~~~~~~\n":
                    checker=True
                if line[:14] == "cpu0_L1D TOTAL":
                    ctr+=1
                    if ctr == 1:
                        prev_misses_l1d = int(line.split()[7])
                        prev_accesses_l1d = int(line.split()[3])
                    if 751 > ctr and ctr > 1:
                        l1d_misses[c][ctr-2] = int(line.split()[7]) - prev_misses_l1d
                        prev_misses_l1d = int(line.split()[7])
                        l1d_accesses[c][ctr-2] = int(line.split()[3]) - prev_accesses_l1d
                        prev_accesses_l1d = int(line.split()[3])
                        if l1d_accesses[c][ctr-2] == 0: continue
                        l1d_small[c][ctr-2] = float(l1d_misses[c][ctr-2])/float(l1d_accesses[c][ctr-2])
                if line[:14] == "cpu0_L2C TOTAL":
                    if ctr == 1:
                        prev_misses_l2c = int(line.split()[7])
                        prev_accesses_l2c = int(line.split()[3])
                    if 751 > ctr and ctr > 1:
                        l2c_misses[c][ctr-2] = int(line.split()[7]) - prev_misses_l2c
                        prev_misses_l2c = int(line.split()[7])
                        l2c_accesses[c][ctr-2] = int(line.split()[3]) - prev_accesses_l2c
                        prev_accesses_l2c = int(line.split()[3])
                        if l2c_accesses[c][ctr-2] == 0: continue
                        l2c_small[c][ctr-2] = float(l2c_misses[c][ctr-2])/float(l2c_accesses[c][ctr-2])
                if line[:9] == "LLC TOTAL":
                    if ctr == 1:
                        prev_misses_llc = int(line.split()[7])
                        prev_accesses_llc = int(line.split()[3])
                    if 751 > ctr and ctr > 1:
                        llc_misses[c][ctr-2] = int(line.split()[7]) - prev_misses_llc
                        prev_misses_llc = int(line.split()[7])
                        llc_accesses[c][ctr-2] = int(line.split()[3]) - prev_accesses_llc
                        prev_accesses_llc = int(line.split()[3])
                        if llc_accesses[c][ctr-2] == 0: continue
                        llc_small[c][ctr-2] = float(llc_misses[c][ctr-2])/float(llc_accesses[c][ctr-2])
            if not checker or ctr != 751:
                print("\n Error:", fname) # Check if a file didn't finish correctly
    print("|",end="")
    #print(l1d_small)
    l1d_small['id'] = file + '_' + l1d_small.index.map(str)
    l2c_small['id'] = file + '_' + l2c_small.index.map(str)
    llc_small['id'] = file + '_' + llc_small.index.map(str)
    l1d_accesses['id'] = file + '_' + l1d_accesses.index.map(str)
    l2c_accesses['id'] = file + '_' + l2c_accesses.index.map(str)
    llc_accesses['id'] = file + '_' + llc_accesses.index.map(str)
    l1d_misses['id'] = file + '_' + l1d_misses.index.map(str)
    l2c_misses['id'] = file + '_' + l2c_misses.index.map(str)
    llc_misses['id'] = file + '_' + llc_misses.index.map(str)
    #print(l1d_small)
    
    l1d_full = l1d_full.append(l1d_small, ignore_index = True)
    l2c_full = l2c_full.append(l2c_small, ignore_index = True)
    llc_full = llc_full.append(llc_small, ignore_index = True)
    #print(file)
    #print(l2c_misses)
    #for c in columns[1:]:
    #    print(l2c_misses[c].sum(), end=", ")
    l1d_accesses_full = l1d_accesses_full.append(l1d_accesses, ignore_index = True)
    l2c_accesses_full = l2c_accesses_full.append(l2c_accesses, ignore_index = True)
    llc_accesses_full = llc_accesses_full.append(llc_accesses, ignore_index = True)
    l1d_misses_full = l1d_misses_full.append(l1d_misses, ignore_index = True)
    l2c_misses_full = l2c_misses_full.append(l2c_misses, ignore_index = True)
    llc_misses_full = llc_misses_full.append(llc_misses, ignore_index = True)

|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||

In [5]:
print(l2c_accesses_full)

                         id    32    64   128   256   512  1024  2048  4096  \
0       400.perlbench-41B_0   994   994   994   994   994   994   994   994   
1       400.perlbench-41B_1  1262  1262  1262  1262  1262  1262  1262  1262   
2       400.perlbench-41B_2   761   761   761   761   761   761   761   761   
3       400.perlbench-41B_3   464   464   464   464   464   464   464   464   
4       400.perlbench-41B_4   831   831   831   831   831   831   831   831   
...                     ...   ...   ...   ...   ...   ...   ...   ...   ...   
141556     657.xz_s-56B_744   838   838   838   838   838   838   838   838   
141557     657.xz_s-56B_745   836   836   836   836   836   836   836   836   
141558     657.xz_s-56B_746   838   838   838   838   838   838   838   838   
141559     657.xz_s-56B_747   836   836   836   836   836   836   836   836   
141560     657.xz_s-56B_748   836   836   836   836   836   836   836   836   

        8192  
0        994  
1       1262  
2     

In [6]:
print(l1d_accesses_full)
print(llc_full)

                         id      32      64     128     256     512    1024  \
0       400.perlbench-41B_0  506969  506971  506970  506970  506969  506967   
1       400.perlbench-41B_1  482431  482431  482430  482431  482431  482431   
2       400.perlbench-41B_2  484623  484623  484622  484622  484624  484623   
3       400.perlbench-41B_3  479864  479864  479864  479864  479865  479864   
4       400.perlbench-41B_4  512599  512598  512598  512600  512599  512598   
...                     ...     ...     ...     ...     ...     ...     ...   
141556     657.xz_s-56B_744  148212  148212  148212  148212  148212  148212   
141557     657.xz_s-56B_745  148175  148175  148175  148175  148175  148175   
141558     657.xz_s-56B_746  148188  148188  148188  148188  148188  148188   
141559     657.xz_s-56B_747  148189  148189  148189  148189  148189  148189   
141560     657.xz_s-56B_748  148152  148152  148152  148152  148152  148152   

          2048    4096    8192  
0       506968  50

In [7]:
l1d_full.to_csv('caches/l1d_miss_ratio.csv',index=False)
l2c_full.to_csv('caches/l2c_miss_ratio.csv',index=False)
llc_full.to_csv('caches/llc_miss_ratio.csv',index=False)
l1d_accesses_full.to_csv('caches/l1d_accesses.csv',index=False)
l2c_accesses_full.to_csv('caches/l2c_accesses.csv',index=False)
llc_accesses_full.to_csv('caches/llc_accesses.csv',index=False)
l1d_misses_full.to_csv('caches/l1d_misses.csv',index=False)
l2c_misses_full.to_csv('caches/l2c_misses.csv',index=False)
llc_misses_full.to_csv('caches/llc_misses.csv',index=False)

In [2]:
import json
import os
import csv
import pandas as pd
import numpy as np

In [13]:
path = '../ce_raw_data/raw_data/'
id_ctr = 0
columns = ['id','32','64','128','256','512','1024','2048','4096','8192']
wb_accesses_full = pd.DataFrame()
wb_misses_full = pd.DataFrame()
for file in os.listdir(path):
    if file[-9:] != '_2048.out':
        continue
    wb_accesses = pd.DataFrame(0., index=np.arange(749), columns=columns)
    wb_misses = pd.DataFrame(0., index=np.arange(749), columns=columns)
    for c in columns[1:]:
        fname = file[:-9] + '_' + c + '.out'
        checker = False
        with open(path+fname) as f:
            lines = f.readlines()
            ctr = 0
            prev_misses_l1d = 0.
            prev_accesses_l1d = 0.
            prev_misses_wb = 0.
            prev_accesses_wb = 0.
            for line in lines:
                if line == "~~~~~~~~~~Reuse Distance Profiler output~~~~~~~~~~\n":
                    checker=True
                if line[:14] == "cpu0_L1D TOTAL":
                    ctr+=1
                if line[:13] == "LLC WRITEBACK":
                    if ctr == 1:
                        prev_misses_wb = int(line.split()[7])
                        prev_accesses_wb = int(line.split()[3])
                    if 751 > ctr and ctr > 1:
                        wb_misses[c][ctr-2] = int(line.split()[7]) - prev_misses_wb
                        prev_misses_wb = int(line.split()[7])
                        wb_accesses[c][ctr-2] = int(line.split()[3]) - prev_accesses_wb
                        prev_accesses_wb = int(line.split()[3])
            if not checker or ctr != 751:
                print("\n Error:", fname) # Check if a file didn't finish correctly
    print("|",end="")
    #print(l1d_small)
    wb_accesses['id'] = file[:-9] + '_' + wb_accesses.index.map(str)
    wb_misses['id'] = file[:-9] + '_' + wb_misses.index.map(str)
    
    #print(l1d_small)
    wb_accesses_full = pd.concat([wb_accesses_full, wb_accesses], ignore_index = True)
    wb_misses_full = pd.concat([wb_misses_full, wb_misses], ignore_index = True)
    print(wb_misses_full)

|                        id     32     64    128    256   512  1024  2048  \
0      400.perlbench-41B_0  142.0  142.0  142.0  141.0  60.0  21.0  19.0   
1      400.perlbench-41B_1  128.0  128.0  128.0  128.0  63.0  22.0   7.0   
2      400.perlbench-41B_2   81.0   81.0   81.0   81.0  53.0  11.0   4.0   
3      400.perlbench-41B_3   52.0   52.0   52.0   52.0  34.0   7.0   4.0   
4      400.perlbench-41B_4   87.0   87.0   87.0   87.0  60.0  12.0   4.0   
..                     ...    ...    ...    ...    ...   ...   ...   ...   
744  400.perlbench-41B_744   39.0   39.0   39.0   39.0  29.0   1.0   0.0   
745  400.perlbench-41B_745   42.0   42.0   42.0   42.0  30.0   1.0   0.0   
746  400.perlbench-41B_746   49.0   49.0   49.0   49.0  39.0   2.0   0.0   
747  400.perlbench-41B_747  117.0  117.0  117.0  116.0  78.0  12.0   2.0   
748  400.perlbench-41B_748    0.0    0.0    0.0    0.0   0.0   0.0   0.0   

     4096  8192  
0     1.0   0.0  
1     2.0   0.0  
2     1.0   0.0  
3     0.0   0.

KeyboardInterrupt: 

In [12]:
wb_accesses_full.to_csv('caches/llc_writeback_accesses.csv',index=False)
wb_misses_full.to_csv('caches/llc_writeback_misses.csv',index=False)

In [1]:
import json
import os
import csv
import pandas as pd
import numpy as np

In [ ]:
path = '../ce_raw_data/raw_data/'
id_ctr = 0
columns = ['id','32','64','128','256','512','1024','2048','4096','8192']
wb_accesses_full = pd.DataFrame()
wb_misses_full = pd.DataFrame()
for file in os.listdir(path):
    if file[-9:] != '_2048.out':
        continue
    wb_accesses = pd.DataFrame(0., index=np.arange(749), columns=columns)
    wb_misses = pd.DataFrame(0., index=np.arange(749), columns=columns)
    for c in columns[1:]:
        fname = file[:-9] + '_' + c + '.out'
        checker = False
        with open(path+fname) as f:
            lines = f.readlines()
            ctr = 0
            prev_misses_l1d = 0.
            prev_accesses_l1d = 0.
            prev_misses_wb = 0.
            prev_accesses_wb = 0.
            for line in lines:
                if line == "~~~~~~~~~~Reuse Distance Profiler output~~~~~~~~~~\n":
                    checker=True
                if line[:14] == "cpu0_L1D TOTAL":
                    ctr+=1
                if line[:13] == "LLC WRITEBACK":
                    if ctr == 1:
                        prev_misses_wb = int(line.split()[7])
                        prev_accesses_wb = int(line.split()[3])
                    if 751 > ctr and ctr > 1:
                        wb_misses[c][ctr-2] = int(line.split()[7]) - prev_misses_wb
                        prev_misses_wb = int(line.split()[7])
                        wb_accesses[c][ctr-2] = int(line.split()[3]) - prev_accesses_wb
                        prev_accesses_wb = int(line.split()[3])
            if not checker or ctr != 751:
                print("\n Error:", fname) # Check if a file didn't finish correctly
    print("|",end="")
    #print(l1d_small)
    wb_accesses['id'] = file[:-9] + '_' + wb_accesses.index.map(str)
    wb_misses['id'] = file[:-9] + '_' + wb_misses.index.map(str)
    
    #print(l1d_small)
    wb_accesses_full = pd.concat([wb_accesses_full, wb_accesses], ignore_index = True)
    wb_misses_full = pd.concat([wb_misses_full, wb_misses], ignore_index = True)
    print(wb_misses_full)